# FIT5196 - Assignment 01

## Group 025

### Group member 1: Raguram Ramakrishnasamy Dhandapani , Student Number: 30151325
### Group member 2: Thirugnanam Ramanathan, Student Number: 30404975

#### Date: 25-08-2019

#### Version: 1.0
Environment: Python 3 and Jupyter Notebook

### Libraries Used: 
<li>pandas 0.19.2 (for data frame, included in Anaconda Python 3.6)</li>
<li>re 2.2.1 (for regular expression, included in Anaconda Python 3.6)</li>

## 1.Introduction

<p>The assignment focuses on extracting the required data from list of multiple XML files stored as a single text file. This program has multiple functions implemented to extract the required data from all of these XML files and is complied into a CSV and JSON file outputs. </p>
<p> The data is parsed from the input file named Group025.txt and the cleaned extracted text is exported to CSV and JSON files </p> 

## 2. Import Libraries

Importing standard libraries:
 
<li>pandas - Working with data frames</li>
<li>re - using regular expressions for parsing text data</li>

In [23]:
import re
import pandas as pd

## 3. Loading data

Standard file reader is used for reading the input file (Group025.txt). Input file is collection of multiple xml documents and file is read into a single string before taken up for parsing. The new line commands are replaced with space for easy processing

In [24]:
#Importing the input file
file_handle = open('Group025.txt','r')

#defining a string to store all the contents of the input file as a single string
complete_string = file_handle.read()

#Replacing the new line syntax with space
complete_string = re.sub('\n',' ',complete_string)


## 4. Functions defined to extract specific details from the individual xml files

### 4.1 Extracting the Inventor names

<li>This function is used to extract all the inventors of the given patent<\li>
<li>It is called from the <b> extract_data </b> function</li>
<li>The function returns the list of all Inventors</li>

In [13]:
#function defined to extract all the inventors for the given patent
def inventor_names(patent):
    # Getting the list of all inventors for the patent
    inventors_extract = re.search('<inventors>(.*)</inventors>',str(patent), flags=16)
    
    inventors = ''
    
    if inventors_extract != '':
        inventors_list = re.finditer('<inventor.*?>(.*?)</inventor>',inventors_extract.group(0), flags=16)
        
        #Extracting individual inventors from the total list
        for item in inventors_list:
            #Extracting First name and Last name of inventors
            if item is not None:
                fn = re.search('<first-name>(.*)</first-name>',item.group(), flags=16)
                ln = re.search('<last-name>(.*)</last-name>',item.group(), flags=16)
                name = ''
                name = re.sub('<(/*)first-name>','',fn.group(0)) + ' '+re.sub('<(/*)last-name>','',ln.group(0))
                
                #appending all names as a single entry
                if inventors == '':
                    inventors = str(name)
                else:
                    inventors = inventors+ ','+ str(name)

        if inventors=='':
            inventors='NA'
    else:
        inventors='NA'

    return '['+inventors+']'

### 4.2. Extracting the Claim text

<li>The function is used to extract claim texts and is called from the extract_data function</li>
<li>The function returns the combined claim text of the given patent xml</li>

In [14]:
#function defined to extract the claim text for the given patent
def claim_text(patent):

    claims = ''
    claims_extract = re.search(r'<claims.*?>(.*?)</claims>',str(patent), flags=16)
    
    #Extracting the total claim text and processing it to get the required texts
    if claims_extract.group(0) != '':
        claims_list = re.findall(r'<claim-text>(.*?)</claim-text>',claims_extract.group(0), flags=16)
        
        for item in claims_list:
            #removing the additional tags present in the claim text
            claimtext = re.sub(r'<.*?>','',str(item))
            
            #Appending claim texts together based on the contents
            if claims == '':
                claims = str(claimtext)
            elif claimtext[0].isdigit() :
                 claims = claims + ', '+ str(claimtext)
            else:
                claims = claims +' '+ str(claimtext)
        
    else:
        claims = 'NA'
    
    #replacing the new line tags in the claim text
    return '['+ re.sub('\n',' ',claims) +']'

### 4.3 Extracting the Abstract

<li>The function is used to extract abstracts of the patent and is called from the extract_data function</li>
<li>The function returns the Abstract of the given patent xml</li>

In [15]:
#function defined to extract the abstract for the given patent
def abstract(patent):
    
    #Object that defines the abstract contents
    abstract_extract = re.search('<abstract .*?>(.*?)</abstract>',str(patent), flags=16)
    abstext='NA'
    
    #if the abstract is present then the contents in it is processed to get the abstract
    if abstract_extract is not None:
        #replacing tags in the abstract
        abstext = re.sub('\s*<.*?>\s*','',abstract_extract.group(0))
        
        if abstext!='[]':  
            abstext = re.sub('[\[\]]','',str(abstext))
        else:
            abstext = 'NA'
            
    return abstext

### 4.4 Extracting the Kind

<li>The function is used to extract the kind of patent and is called from the extract_data function</li>
<li>The function returns the Kind of the given patent xml</li>
<li>The patent kind is identified as shown below and the <b>reference for it is taken from the Sample input and output </b> </li>

In [16]:
#function defined to extract the kind for the given patent
def kind_extract(patent):

    #based on the application type of the patent, we classify the kind of patent
    kindextract = re.findall('appl-type="(.*?)">',str(patent), flags=16)[0]
    
    #Using the publication reference of the patent to identify the publication
    kindtype =  re.findall('<publication-reference>(.*?)</publication-reference>',str(patent), flags=16)
    kind = re.findall('<kind>(.*?)</kind>',str(kindtype), flags=16)[0]
    kindtext=''
    
    #Based on the type of the kind, the corresponding kind is allocated
    if kindextract != '[]':
        if kindextract == 'utility' and kind == 'B2':
            kindtext = 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.'
        elif kindextract == 'utility' and kind == 'B1':
            kindtext = 'Utility Patent Grant (no published application) issued on or after January 2, 2001.'
        elif kindextract == 'plant' and kind == 'P2':
            kindtext = 'Plant Patent Grant (no published application) issued on or after January 2, 2001.'
        elif kindextract == 'plant' and kind == 'P3':
            kindtext = 'Plant Patent Grant (with a published application) issued on or after January 2, 2001.'            
        elif kindextract == 'design':
            kindtext = 'Design Patent'        
        elif kindextract == 'reissue':
            kindtext = 'Reissue Patent'

        return kindtext
    else:
        return 'NA'

## 5. Main function for Extracting details by calling other functions 

<li> This function is called while iterating the XML Tags</li>
<li> Grant id, Title name and number of claims are extracted using a regex expression directly</li>
<li> Inventors is found by calling the inventor_names(patent) function</li>
<li> citation application count and citation examiner count are extracted using a regex expression directly</li>
<li> claims_text, abstract_text and kind is found by calling the claim_text(patent), abstract(patent) and kind_extract(patent) function respectively</li>
<li> This function returns the item list that contains every detail of the patent xml</li>

In [17]:
def extract_data(patent):
    #Fucntion defined to extract data for each individual patents
    
    item_list = []
    
    #Extracting the individual items of the patent one by one by calling the corresponding function whereever necessary
    grant_id = re.findall('file="(.*?)-',str(patent), flags=16)[0]
    title_name = re.findall('<invention-title id="[\w]*">(.*?)</invention-title>',str(patent),flags=16)[0]
    number_of_claims = int(re.findall('<number-of-claims>(.*)</number-of-claims>',str(patent),flags=16)[0])
    inventors = inventor_names(patent)
    citations_applicant_count  = len(re.findall('(<category>cited by applicant</category>)', str(patent), flags=16))
    citation_examiner_count = len(re.findall('(<category>cited by examiner</category>)', str(patent), flags=16))
    claims_text  = claim_text(patent)
    abstract_text = abstract(patent)
    kind = kind_extract(patent)

    item_list = [grant_id, title_name,kind, number_of_claims, inventors, citations_applicant_count, citation_examiner_count, claims_text,abstract_text]
    
    return item_list

## 6. Code for iteration of XML tags

<li>Each of the xml tags is iterated for abstracting the data in it.</li>
<li>Main function extract_data is called on each iteration.</li>
<li>The data extracted from the function is saved as dictionary with key as Grant ID and other details as list of values.</li>

In [25]:
#Using re, the text file is split into multiple XML files
xml_list = re.finditer('<\?xml version="1.0" encoding="UTF-8"\?>(.*?)(?=<\?xml version="1.0" encoding="UTF-8"\?>)|<\?xml version="1.0" encoding="UTF-8"\?>(.*)', complete_string, flags=16)

#the above object is used to iterate over the list of all XML files and the required data is extracted for each of them

#The dict used to store all the patent details written to CSV
patdict={}
#the dict used to store the patent details that is written to JSON
patdict_json = {}
#Column tags used in CSV and JSON
column_tags = ['patent_title','kind','number_of_claims','inventors','citations_applicant_count','citations_examiner_count','claims_text','abstract']

#scanning through all the individual patents
for item in xml_list:
    
    #the item_list contains the list of the information of a patent
    item_list = extract_data(item.group())
    #Assigning data to CSV dict with Grant_ID as key
    patdict[item_list[0]] = item_list[1:]
    
    #Defining a dict to store the values dictionary of the JSON file
    values_dict = {}
    for i in range(len(column_tags)):
        values_dict[column_tags[i]]=item_list[i+1]
    #the items added to the JSON dict with grant ID as key and values_dict as Values
    patdict_json[item_list[0]] = values_dict


## 7.Creating Dataframe from Dictionary

<li> Dictionary is converted to dataframe using pandas.</li>
<li> All coloumn names are set to the data frame with index name as Grant id. </li>

In [26]:
#Pandas used to create the Dataframe from the dictionary
df = pd.DataFrame.from_dict(patdict, orient='index',)
#Column names written to the DF
df.columns = ['patent_title','kind','number_of_claims','inventors','citations_applicant_count','citations_examiner_count','claims_text','abstract']
df.index.name = 'grant_id'
df.head()

patent_title  \
grant_id                                                        
US10362338                                   Image processing   
US10360934                                    Carriage device   
US10362219                        Avatar creation and editing   
US10362018  Downloadable security and protection methods a...   
US10358273      Head stand closure with a tamper-evident seal   

                                                         kind  \
grant_id                                                        
US10362338  Utility Patent Grant (with a published applica...   
US10360934  Utility Patent Grant (with a published applica...   
US10362219  Utility Patent Grant (with a published applica...   
US10362018  Utility Patent Grant (with a published applica...   
US10358273  Utility Patent Grant (with a published applica...   

            number_of_claims  \
grant_id                       
US10362338                20   
US10360934                 8   
US10362219                13   
US10362018                20   
US10358273                12   

                                                    inventors  \
grant_id                                                        
US10362338                             [Viacheslav Chesnokov]   
US10360934  [Atsushi Miyahara,Kazuo Ohyama,Keisei Hakamata...   
US10362219  [Christopher Wilson,Guillaume Barlier,Sebastia...   
US10362018  [William L. Helms,John B. Carlucci,Jason Kazmi...   
US10358273              [Claude Benoit-Gonin,Philippe Bardet]   

            citations_applicant_count  citations_examiner_count  \
grant_id                                                          
US10362338                          4                         4   
US10360934                          3                         9   
US10362219                        223                         3   
US10362018                        536                         2   
US10358273                          4                         6   

                                                  claims_text  \
grant_id                                                        
US10362338  [1. A method of processing an image comprising...   
US10360934  [1. A recording apparatus comprising: a carria...   
US10362219  [1. An electronic device, comprising: a displa...   
US10362018  [1. A computerized method of operating a secur...   
US10358273  [1. A head stand container closure, the closur...   

                                                     abstract  
grant_id                                                       
US10362338  A method of processing an image is provided. T...  
US10360934  A carriage device includes a first guide membe...  
US10362219  The present disclosure generally relates to cr...  
US10362018  Methods and apparatus for control of data and ...  
US10358273  A head stand container closure is provided. Th...

## 8.Writing to CSV 

<li>Pandas is used for converting the dataframe into a csv file (one of the required output files).</li>

In [27]:
#storing data to csv file
pd.DataFrame.to_csv(df, 'Group025.csv')

## 9.Writing to JSON

<li>The dictionary is converted into a Complete string to write as JSON file</li>
<li>The single quote in dictionary is replaced as double quotes to make the dict as a proper JSON formatted content</li>
<li>This converted string is then written into the JSON file</li>

In [28]:
#Variable defined to convert the dict to str
JSON_dict = str(patdict_json)

#replacing the required contents in the file, to make it JSON ready
replace_dict ={"{\'": '{"', "\':" : '":', ": \'":': "', "\', \'" :'", "', "\'}": '"}', ", \'" :', "'}
for ke,val in replace_dict.items():
    JSON_dict = JSON_dict.replace(ke,val)

#Writing the file to the output JSON FIle
fo = open('Group025.json', 'w')
fo.write(JSON_dict)
fo.close()

## 10.Error checking 

<li>Sample input is loaded as input file and the obtained output is compared with the sample output</li>
<li>The compared results(csv) has zero difference when checked using formula in excel</li>
<li>The output JSON file is compared in the code-beautify JSON viewer and also checked by loading the output back as dataframe.</li>

## 11.Summary

This assignment helps in understanding the basic text file parsing techniques using Python language (re package). 
The outcomes of these assignment are:

**XML parsing and data extraction:** 
<p> By using the regular expressions package and basic functions, we were able to parse the data and extract them to dictionaries to convert it to required outputs </p>

**Data frame manipulation:** 
<p> By using the pandas package, we have loaded the dictionaries to data frame </p>

**Exporting data to specific format:**
<p> By using built-in functions like 'to_csv()' it was possible to export data frames into .csv files. The parsed value in lists are converted into dictionary with patent id as key of the dictionary. The value for each key are provided as dictionary with parsed value title as key. Using the basic file operations like write(), it is written into the json file. </p>
